In [ ]:
import os
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
path= '/content/drive/MyDrive/news dataset/children stories/scraped/'
os.chdir(path)

In [ ]:
train_df = pd.read_csv('dataset_with_keys_30a.csv')

In [ ]:
train_df[:10]

,title,story,class,url,keywords,key_scores
0,On the go with Zibo,zibo is a little zebra. you know what a zebra ...,adventure,https://www.bedtime.com/on-the-go-with-zibo/,"['zibo', 'animal', 'give', 'zebra', 'cow', 'st...","[0.025387348146749516, 0.09251316835653439, 0...."
1,A Walk in the Woods,he tapped the surface of the pool with the for...,adventure,https://www.bedtime.com/a-walk-in-the-woods/,"['tapped the surface', 'mother', 'back', 'mud'...","[0.027205449786734906, 0.08402403176673347, 0...."
2,"A penguin, A black egg and an island","at dawn i heard a knock on the door, it was sa...",adventure,https://www.bedtime.com/a-penguin-a-island-and...,"['leo', 'dread clear dragon', 'princess laytin...","[0.019581578541566145, 0.031555784992482, 0.03..."
3,To the Princess,"some years ago, i was a young man and i was on...",adventure,https://www.bedtime.com/to-the-princess/,"['tim', 'dark lords', 'ill', 'james', 'dark', ...","[0.010967980872248788, 0.02461408568559992, 0...."
4,The Ultimate Quest,"urrggh, owen grunted, he was in the castle doi...",adventure,https://www.bedtime.com/the-ultimate-quest/,"['owen', 'chippie', 'bolin', 'asked owen', 'br...","[0.005754948475371684, 0.01732583333168135, 0...."
5,The Crossbow of the Wizard,"my name is corrin, the warriorhero of my natio...",adventure,https://www.bedtime.com/the-crossbow-of-the-wi...,"['ragnarok', 'sans', 'dread spell', 'electrani...","[0.026467812207965406, 0.05747927504750636, 0...."
6,Harry's magic pirate ship,"harrys magic pirate ship, there was once a lit...",adventure,https://www.bedtime.com/harrys-magic-pirate-ship/,"['pirate ship', 'ship', 'pirate', 'harry', 'gr...","[0.00251343500247789, 0.007691822199021019, 0...."
7,PUGSLEY,", joe liked hikes. and this one was going to ...",adventure,https://www.bedtime.com/pugsley/,"['joe', 'pastor maxwell', 'maxwell', 'pugsley'...","[0.010544564386321037, 0.019892467037892263, 0..."
8,Jenny's Surprise,this is jenny. jenny lives in a pretty red ho...,adventure,https://www.bedtime.com/jennys-surprise/,"['jenny', 'carol', 'ducks', 'basket', 'carol.'...","[0.02878686262803785, 0.05881630101724026, 0.0..."
9,The Wild West Chase,now what? asked ivy as she buried her head ins...,adventure,https://www.bedtime.com/the-wild-west-chase/,"['ivy', 'karen', 'horses', 'rifle', 'back', 'j...","[0.013279730197117421, 0.014956584019622283, 0..."


In [ ]:
len(np.unique(train_df['class']))

13

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [ ]:
# Load the saved model
# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')
# Load the saved model weights into the initialized model
model.load_state_dict(torch.load('T5_story_gen_key_30a.bin',map_location=torch.device('cpu')))
model=model.to(device)

In [ ]:
train_df.iloc[1]['title']

'A Walk in the Woods'

In [ ]:
test_df=train_df[280:]
predicted_text=[]
for i in range(len(test_df)):
    title= test_df.iloc[i]['title']
    key=test_df.iloc[i]['keywords']
    cls=test_df.iloc[i]['class']
    # Set the input string
    input_str="<title> "+ title + " <keywords> " + ",".join(key) +'<class> ' + cls
    max_length=300
    # Tokenize the input string
    input_ids = tokenizer.encode(input_str, return_tensors='pt')
    input_ids = input_ids.to(device)
    # Generate the output
    output_ids = model.generate(input_ids,max_length=max_length)

    # Decode the output
    output_str = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predicted_text.append(output_str)
    # Print the output
    #(output_str)

Token indices sequence length is longer than the specified maximum sequence length for this model (1190 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding
print(locale.getpreferredencoding())

UTF-8


In [ ]:
# !pip install rouge

In [ ]:
from rouge import Rouge
roughe_scores=[]
for i in range(len(predicted_text)):
    reference = test_df.iloc[i]['story']
    candidate = predicted_text[i]
    rouge = Rouge()
    scores = rouge.get_scores(candidate, reference)
    rouge_score = scores[0]['rouge-l']['f']
    print(rouge_score)
    roughe_scores.append(rouge_score)

0.23655913527112968
0.16144974911252313
0.2509505659809308
0.26055045447932
0.22119815211694888
0.28930817123432623
0.1893491074262107
0.17989417578256184
0.2103658498041066
0.3009404339540689
0.06349206166671711
0.19393938915169895
0.2166666616791668
0.24233128462442138
0.2586206854643943
0.24164523934708343
0.2499999951652893
0.2812499952870009
0.2737642535552054
0.2959641209723703
0.2013129056010803
0.2307692271345046
0.19607842714148416


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu
belu_scores=[]
for i in range(len(predicted_text)):
    reference = [test_df.iloc[i]['story']]
    candidate = predicted_text[i] 
    self_bleu = sentence_bleu(reference, candidate)
    #print('self_bleu---',self_bleu)
    belu_scores.append(self_bleu)


In [ ]:
print('20b - Keywords')
average_belu_score= np.sum(belu_scores)/len(belu_scores)
print('Average Bleu Score:',average_belu_score)
avrage_rough_score=np.sum(roughe_scores)/len(roughe_scores)
print('Average Rough Score:',avrage_rough_score)


20b - Keywords
Average Bleu Score: 0.25107623063655177
Average Rough Score: 0.22810434965011056


In [ ]:
i=0
print(test_df.iloc[i]['class'])
predicted_text[i]

Sabrina


"title> chapter one dindra leonord ran down old and new yorkshire. sheep were flocking from northeastern strife trees in the jungle. fourty masons, two yas, and two peddlers were all in search of healthy food. one of the shelters I spoke was that the people in the forest cared for them would face a harsh and harsh weather conditions. the animals welfare groups said there was a high risk factor in the situation. the coucous were very very happy Sabrina, his mother said in an owl, his mother said, her loving wife, all we blackened out the parcel and skimmeted the forests of old barn for five more minutes. do you know what the reason, car he called. do you think the three of us want to get the best breakfast in the forest? isnt this food is very good. do you know what you are looking for, sauntered for the unexpected. if you dont work in the forest? automakers started to draw out the suggested foodie packets? the public could ask invad. as the public could question the public. oh, well, j

In [ ]:
pip install language_tool_python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import language_tool_python

def calculate_grammar_score(text):
    tool = language_tool_python.LanguageTool('en-US')
    matches = tool.check(text)
    score = 1 - len(matches) / len(text.split())
    return score
GRAM=[]
for i in range(len(predicted_text)):
  score = calculate_grammar_score(predicted_text[i])
  print("Grammar Score:", score)
  GRAM.append(score)

0.8659777403785307

In [ ]:
print('20b')
import numpy as np
gram_score=np.sum(GRAM)/len(GRAM)
gram_score
